<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Fake-news-detection" data-toc-modified-id="Fake-news-detection-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Fake news detection</a></span><ul class="toc-item"><li><span><a href="#Feature-extraction-functions" data-toc-modified-id="Feature-extraction-functions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Feature extraction functions</a></span></li><li><span><a href="#Reading-and-preparing-the-corpus" data-toc-modified-id="Reading-and-preparing-the-corpus-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Reading and preparing the corpus</a></span></li><li><span><a href="#Parametrization-and-feature-extraction" data-toc-modified-id="Parametrization-and-feature-extraction-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Parametrization and feature extraction</a></span><ul class="toc-item"><li><span><a href="#Generating-N-gram-Feature-Combinations" data-toc-modified-id="Generating-N-gram-Feature-Combinations-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Generating N-gram Feature Combinations</a></span></li></ul></li><li><span><a href="#Frequency-threshold" data-toc-modified-id="Frequency-threshold-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Frequency threshold</a></span></li><li><span><a href="#Weighting-schemes" data-toc-modified-id="Weighting-schemes-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Weighting schemes</a></span></li><li><span><a href="#Classification-Process" data-toc-modified-id="Classification-Process-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Classification Process</a></span><ul class="toc-item"><li><span><a href="#Test-Prediction-and-Result-Evaluating-Fucntion" data-toc-modified-id="Test-Prediction-and-Result-Evaluating-Fucntion-1.6.1"><span class="toc-item-num">1.6.1&nbsp;&nbsp;</span>Test Prediction and Result Evaluating Fucntion</a></span></li><li><span><a href="#Initializing-classification-algorithms" data-toc-modified-id="Initializing-classification-algorithms-1.6.2"><span class="toc-item-num">1.6.2&nbsp;&nbsp;</span>Initializing classification algorithms</a></span></li><li><span><a href="#Cross-Validation-Classification-and-Evaluation-Function" data-toc-modified-id="Cross-Validation-Classification-and-Evaluation-Function-1.6.3"><span class="toc-item-num">1.6.3&nbsp;&nbsp;</span>Cross-Validation Classification and Evaluation Function</a></span></li></ul></li><li><span><a href="#End-to-End-Pipeline" data-toc-modified-id="End-to-End-Pipeline-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>End-to-End Pipeline</a></span><ul class="toc-item"><li><span><a href="#Running-the-Experiment" data-toc-modified-id="Running-the-Experiment-1.7.1"><span class="toc-item-num">1.7.1&nbsp;&nbsp;</span>Running the Experiment</a></span></li></ul></li><li><span><a href="#Additional-Experiments-for-Paper" data-toc-modified-id="Additional-Experiments-for-Paper-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Additional Experiments for Paper</a></span><ul class="toc-item"><li><span><a href="#ROC-values-for-LinerSVC" data-toc-modified-id="ROC-values-for-LinerSVC-1.8.1"><span class="toc-item-num">1.8.1&nbsp;&nbsp;</span>ROC-values for LinerSVC</a></span></li><li><span><a href="#ROC-curves" data-toc-modified-id="ROC-curves-1.8.2"><span class="toc-item-num">1.8.2&nbsp;&nbsp;</span>ROC-curves</a></span></li></ul></li></ul></li></ul></div>

# Fake news detection

In [23]:
## creator ????
## modify: omar edgardo lugo sanchez
## emanil: lugo.oe@gmail.com

In [24]:
# needed to make it run on Alisa's laptop -- ignore otherwise 
import sys 
#sys.executable, sys.path
sys.path.append('/Users/alisa/workspace/Urdu_Fake_News/envname/lib/python3.7/site-packages')
import re
import glob
import numpy as np
import os
import json
import argparse
import time
import codecs
import string
import codecs
import random
import scipy.sparse as sp
from pathlib import Path
# roc curve and auc
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from random import randrange
from scipy.sparse import csr_matrix, csc_matrix, hstack, coo_matrix
from gensim.matutils import Scipy2Corpus, corpus2csc
from gensim.models.logentropy_model import LogEntropyModel
from collections import defaultdict, Counter
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, make_scorer, accuracy_score
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords as sw
from string import punctuation
from sklearn.pipeline import Pipeline
import pandas as pd 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.ensemble.weight_boosting import AdaBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from IPython.display import display
from sklearn.metrics import precision_recall_fscore_support

In [25]:
ngram_feat_combinations = []

for i in range(3,7):
    for j in range(0,7):
        for k in range(0,7):
            ngram_feat_combinations.append([i,j,k])
        
del ngram_feat_combinations[0]
len(ngram_feat_combinations)

342

In [26]:
def preprocessText(text):
    #here remove text
    cleantext=re.sub("\d+", "0", text)
    return cleantext
#utility function for reading files
def read_txt_files(files):
    text=[]
    topic=[]
    for i,file_path in enumerate(files):
        with open(file_path,'r') as infile:
            cleantext=preprocessText(infile.read())
            text.append(cleantext)
            file_topic=''.join(re.findall('[A-Za-z]', file_path.stem))
            topic.append(file_topic)
    return text, topic

In [27]:
import re, string, timeit
def len_text_no_punc(s):
    string_w = s.translate(str.maketrans('','',string.punctuation)) 
    return len(string_w)

In [28]:
# load train data
train_real_news, train_real_news_topics = read_txt_files(sorted(Path('Corpus/Train/Real/').glob('*.txt')))
train_fake_news, train_fake_news_topics = read_txt_files(sorted(Path('Corpus/Train/Fake/').glob('*.txt')))
real = [1] * len(train_real_news_topics)
fake = [0] * len(train_fake_news_topics)
d = {'Text': train_real_news, 'Topic': train_real_news_topics,'Class':  real }
df1 = pd.DataFrame(data=d)
d = {'Text': train_fake_news, 'Topic': train_fake_news_topics,'Class': fake}
df2 = pd.DataFrame(data=d)
Train_data = pd.concat([df1,df2])
#load test data
real_news, real_news_topics = read_txt_files(sorted(Path('Corpus/Test/Real/').glob('*.txt')))
fake_news, fake_news_topics = read_txt_files(sorted(Path('Corpus/Test/Fake/').glob('*.txt')))
real = [1] * len(real_news_topics)
fake = [0] * len(fake_news_topics)
d = {'Text': real_news, 'Topic': real_news_topics,'Class': real}
df1 = pd.DataFrame(data=d)
d = {'Text': fake_news, 'Topic': fake_news_topics,'Class': fake}
df2 = pd.DataFrame(data=d)
Test_data = pd.concat([df1,df2])
df = pd.concat([Train_data,Test_data])
df['lengh_text'] = df['Text'].apply(len_text_no_punc)
df = df.sort_values(by=['lengh_text'], ascending=True)
df = df.reset_index(drop=True)
df = df[df['lengh_text']>=6] # only keep text with len more than 6 because n char
df.head(10)

,Text,Topic,Class,lengh_text
1,نئی دہلی 0 ڈسمبر(ایجنسی) عالمی بازار کے مثبت س...,bus,1,308
2,﻿\nآنجہانی برطانوی رائٹر کے Jodi Picoltناول ...,sbz,0,313
3,شیئر کریں:\n\nآنجہانی برطانوی رائٹرP. L. Trav...,sbz,1,326
4,﻿\nلوکا گواڈنینو کی پروڈکشن میں بننے والی فلم ...,sbz,0,332
5,\nنئی دہلی 0 ڈسمبر(ایجنسی) عالمی منڈی اور شادی...,bus,0,334
6,آئی سی سی کا بڑا اعلان\nڈھاکہ 0 دسمبر (سیاست ڈ...,sp,0,363
7,ممبئی(شوبزڈ یسک) بالی وڈ کے معروف اداکار دھرمن...,sbz,1,372
8,مصر 0 دسمبر(ایجنسی) ایک اداکارہ کے مبینہ طور س...,sbz,1,380
9,\nمصر 0 دسمبر(ایجنسی) ایک اداکارہ کے بوائے فری...,sbz,0,382
10,﻿کراچی (شوبزڈ یسک) لالی وڈ کے معروف اداکار دھر...,sbz,0,383


In [29]:
df = pd.concat([df, pd.get_dummies(df['Topic'])], axis=1)  
classe = df['Class']
df = df.drop(["Class","Topic"], axis=1)
df = pd.concat([df,classe], axis=1)
df.head(10)

,Text,lengh_text,bus,hlth,sbz,sp,tch,Class
1,نئی دہلی 0 ڈسمبر(ایجنسی) عالمی بازار کے مثبت س...,308,1,0,0,0,0,1
2,﻿\nآنجہانی برطانوی رائٹر کے Jodi Picoltناول ...,313,0,0,1,0,0,0
3,شیئر کریں:\n\nآنجہانی برطانوی رائٹرP. L. Trav...,326,0,0,1,0,0,1
4,﻿\nلوکا گواڈنینو کی پروڈکشن میں بننے والی فلم ...,332,0,0,1,0,0,0
5,\nنئی دہلی 0 ڈسمبر(ایجنسی) عالمی منڈی اور شادی...,334,1,0,0,0,0,0
6,آئی سی سی کا بڑا اعلان\nڈھاکہ 0 دسمبر (سیاست ڈ...,363,0,0,0,1,0,0
7,ممبئی(شوبزڈ یسک) بالی وڈ کے معروف اداکار دھرمن...,372,0,0,1,0,0,1
8,مصر 0 دسمبر(ایجنسی) ایک اداکارہ کے مبینہ طور س...,380,0,0,1,0,0,1
9,\nمصر 0 دسمبر(ایجنسی) ایک اداکارہ کے بوائے فری...,382,0,0,1,0,0,0
10,﻿کراچی (شوبزڈ یسک) لالی وڈ کے معروف اداکار دھر...,383,0,0,1,0,0,0


In [30]:
x = df[['lengh_text']].values.astype(float)
min_max_scaler = preprocessing.StandardScaler()
df['lengh_text'] = min_max_scaler.fit_transform(x)
df.head(10)

,Text,lengh_text,bus,hlth,sbz,sp,tch,Class
1,نئی دہلی 0 ڈسمبر(ایجنسی) عالمی بازار کے مثبت س...,-1.058687,1,0,0,0,0,1
2,﻿\nآنجہانی برطانوی رائٹر کے Jodi Picoltناول ...,-1.054561,0,0,1,0,0,0
3,شیئر کریں:\n\nآنجہانی برطانوی رائٹرP. L. Trav...,-1.043833,0,0,1,0,0,1
4,﻿\nلوکا گواڈنینو کی پروڈکشن میں بننے والی فلم ...,-1.038882,0,0,1,0,0,0
5,\nنئی دہلی 0 ڈسمبر(ایجنسی) عالمی منڈی اور شادی...,-1.037231,1,0,0,0,0,0
6,آئی سی سی کا بڑا اعلان\nڈھاکہ 0 دسمبر (سیاست ڈ...,-1.013300,0,0,0,1,0,0
7,ممبئی(شوبزڈ یسک) بالی وڈ کے معروف اداکار دھرمن...,-1.005873,0,0,1,0,0,1
8,مصر 0 دسمبر(ایجنسی) ایک اداکارہ کے مبینہ طور س...,-0.999271,0,0,1,0,0,1
9,\nمصر 0 دسمبر(ایجنسی) ایک اداکارہ کے بوائے فری...,-0.997621,0,0,1,0,0,0
10,﻿کراچی (شوبزڈ یسک) لالی وڈ کے معروف اداکار دھر...,-0.996796,0,0,1,0,0,0


In [31]:
xTrain, xTest, yTrain, yTest = train_test_split(df.loc[:, df.columns != 'Class'], df.loc[:, df.columns == 'Class'], test_size = 0.35, random_state = 0)
xTrain.head(10)

,Text,lengh_text,bus,hlth,sbz,sp,tch
472,اس ترقی یافتہ دور میں سائنس دانوں کے ساتھ طالب...,-0.281331,0,0,0,0,1
199,لیمو کا رس یا لیمو کا جوس گرمی کے موسم کا سب س...,-0.626272,0,1,0,0,0
694,کراچی (ویب ڈیسک) وزیر اعلیٰ سندھ مراد علی شاہ ...,0.325205,0,1,0,0,0
822,سابق وزیر اطلاعات اور ترجمان پاکستان مسلم لیگ ...,1.428523,1,0,0,0,0
533,نوجوان محبوب نے اپنی روسی محبوبہ کو علاج کے لی...,-0.177353,0,1,0,0,0
799,﻿بنیادی حقائق\n\nعلم نجوم میں دائرۃ البروج کے0...,1.091834,0,0,1,0,0
22,سندھ کے ضلع تھر پارکر کے علاقے مٹھی میں واقع س...,-0.939856,0,1,0,0,0
581,بوسٹن(ویب ڈیسک) ہمارے دماغ کا ہر خلیہ ایک چھوٹ...,-0.061823,0,0,0,0,1
647,برکلے، کیلیفورنیا انار وہ خوش ذائقہ اور خوبصو...,0.107347,0,1,0,0,0
251,﻿چار مرتبہ اکیڈمی ایوارڈز (آسکر) جیتنے والی پ...,-0.538799,0,0,1,0,0


In [32]:
yTrain['Class'].value_counts()

1    327
0    257
Name: Class, dtype: int64

In [33]:
yTest['Class'].value_counts()

1    172
0    143
Name: Class, dtype: int64

In [34]:
# the class is required to enable proper use of the feature extraction functionality in the experiment pipeline
# how to use fit fit_tranform tranform 
# https://datascience.stackexchange.com/questions/12321/difference-between-fit-and-fit-transform-in-scikit-learn-models

class FeatureExtractor: 
    """Feature extraction"""
    
    def __init__(self, cnvalues, wnvalues, fnvalues): 
        self.cnvalues = cnvalues
        self.wnvalues = wnvalues
        self.fnvalues = fnvalues
        self.vectorizer = CountVectorizer(lowercase=False, min_df=2, tokenizer=lambda x: x.split('&%$')) #--> we can change this
    
    def process_texts(self,texts,cn,wn,fn):
        occurrences=defaultdict(int)
        featuresList=[]
        featuresDict=Counter()
        for (text) in texts:
            features=self.extract_features(text,cn,wn,fn)
            featuresDict.update(features)
            featuresList.append('&%$'.join(features))
        return featuresList, featuresDict
       
    def fit_extract(self, train_texts):
        train_features, dicOfFeatures = self.process_texts(train_texts, self.cnvalues, self.wnvalues, self.fnvalues)
        train_data = self.vectorizer.fit_transform(train_features)
        train_data = train_data.astype(float)
        return train_data, dicOfFeatures
    
    def extract(self, test_texts): 
        test_features, dicOfFeaturesTest = self.process_texts(test_texts, self.cnvalues, self.wnvalues, self.fnvalues)
        test_data = self.vectorizer.transform(test_features)
        test_data = test_data.astype(float)
        return test_data, dicOfFeaturesTest 
    
    def wordNgrams(self,text, n):
        ngrams = []
        text = [word for word in text.split() if word not in string.punctuation]
        ngrams = [' '.join(text[i:i+n])+'' for i in range(len(text)-n+1)]
        return ngrams
    
    def charNgrams(self,text, n):
        ngrams = []
        ngrams = [text[i:i+n]+'_cng' for i in range(len(text)-n+1)]
        return ngrams
    #Extracts function words n-grams with a pre-loaded dictionary
    def funcNgrams(self,text, n):
        stop_words = self.load_diccionario('stop_words.txt')
        patt=r'\b(' + ('|'.join(re.escape(key) for key in stop_words)).lstrip('|') + r')\b'
        pattern = re.compile(patt)
        text = re.sub(r"(\n+|\r+|(\r\n)+)", " ", text)
        text = re.sub(r" +", " ", text)
        text = re.sub(r"’", "'", text)
        text = re.sub(r"[" + punctuation + "]*", "", text)
        terms = pattern.findall(text)
        n_grams=[('_'.join(terms[i:i+n])) + "_fwn" for i in range(len(terms)-n+1)]
        return n_grams
    
    def extract_features(self,text,cn,wn,fn):
        text = text.lower()
        features = []
        if(cn>0):
            features.extend(self.charNgrams(text,cn))
        if(wn>0):
            features.extend(self.wordNgrams(text,wn))
        if(fn>0):
            features.extend(self.funcNgrams(text,fn))
        return features
    
    # Extracts all features in a set of 'texts' and return as a string separated with the simbol '&%$'

    def load_diccionario(self,ruta):
        terms = set()#Dictionary of slangs
        try:
            tmp = open(ruta, "r")     
            while True :
                linea = tmp.readline()                                                                                   
                #linea = to_unicode(linea) 
                if (not linea) or (linea == ""):                                                                               
                    break;                                                                                                      
                linea = linea.rstrip()
                terms.add(linea.lower())
            return (terms)
        except IOError as e:
            print ("Error: "+ruta+" I/O error({0}): {1}".format(e.errno, e.strerror))
            exit(1)
            
    def apply_frequency_threshold(self,feature_mx, N=5):
        values=np.array(feature_mx.sum(axis=0)).ravel()
        thresholdMask=(values >= N)*1
        indices_zero = list(np.nonzero(thresholdMask == 0)[0])
        all_cols = np.arange(feature_mx.shape[1])
        cols_to_keep = np.where(np.logical_not(np.in1d(all_cols, indices_zero)))[0]
        return cols_to_keep 


## Weighting schemes

In [35]:
from gensim.matutils import Scipy2Corpus, corpus2csc
from gensim.models.logentropy_model import LogEntropyModel

class LogEntWeightScheme: 
        
    def __init__(self): 
        self.log_ent = None
        self.train_data_shape = 1
        
    def fit(self, scaled_train_data, *_):
        Xc = Scipy2Corpus(scaled_train_data)
        self.log_ent = LogEntropyModel(Xc)
        X = self.log_ent[Xc]
        X = corpus2csc(X)
        scaled_train_data = sp.csc_matrix.transpose(X)
        self.train_data_shape = scaled_train_data.shape[1]
        return scaled_train_data
        
    def fit_transform(self, scaled_train_data, *_):    
        Xc = Scipy2Corpus(scaled_train_data)
        self.log_ent = LogEntropyModel(Xc)
        X = self.log_ent[Xc]
        X = corpus2csc(X)
        scaled_train_data = sp.csc_matrix.transpose(X)
        self.train_data_shape = scaled_train_data.shape[1]
        return scaled_train_data 

    def transform(self, scaled_test_data, *_):
        Xtest = Scipy2Corpus(scaled_test_data)
        X = self.log_ent[Xtest]
        X = corpus2csc(X, self.train_data_shape)
        scaled_test_data = sp.csc_matrix.transpose(X)
        return scaled_test_data
    
from scipy.sparse import coo_matrix

class RelativeWeightScheme:
    
    def __init__(self):
        self.s = 1.0
             
    def fit(self, scaled_train_data, *_):
        self.s = scaled_train_data.sum(axis=0)
        if(self.s==0):
            self.s=1
        scaled_train_data = coo_matrix(np.nan_to_num(scaled_train_data/self.s), dtype=np.float64)
        return scaled_train_data

    # identical to fit()
    def fit_transform(self, scaled_train_data, *_):
        self.s = scaled_train_data.sum(axis=0)
        if(self.s==0):
            self.s=1        
        scaled_train_data = coo_matrix(np.nan_to_num(scaled_train_data/self.s), dtype=np.float64)
        return scaled_train_data
                
    def transform(self, scaled_test_data, *_):          
        scaled_test_data = coo_matrix(np.nan_to_num(scaled_test_data/self.s), dtype=np.float64)
        return scaled_test_data  
    
class DoNothingWeightScheme:
    """Making it easier to comply with APIs for a do-nothing trasnformer"""

    def __init__(self):
        pass
    
    def fit(self, scaled_train_data, *_):
        return scaled_train_data

    def fit_transform(self, scaled_train_data, *_):
        return scaled_train_data

    def transform(self, scaled_test_data, *_):
        return scaled_test_data
    
def get_weighting_scheme(feature_weight_name): 
    
    if feature_weight_name == 'binary':
        #print ("feature_weight = binary")
        return preprocessing.Binarizer()

    elif feature_weight_name == 'logent':
        #print ("feature_weight = logent")
        return LogEntWeightScheme()
        
    elif feature_weight_name == 'tfidf':
        #print ("feature_weight = tfidf")
        return TfidfTransformer()

    elif feature_weight_name == 'norm':
        # cannot normalize with l2 in a Pipeline
        #print ("feature_weight = norm")
        return preprocessing.MaxAbsScaler()

    elif feature_weight_name == 'relat':
        #print ("feature_weight = relat")
        return RelativeWeightScheme()

    else:
        #print ("feature_weight = tf")
        return DoNothingWeightScheme()         

In [49]:
#https://stackoverflow.com/questions/25009284/how-to-plot-roc-curve-in-python
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import VarianceThreshold
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
valor = 1
clfLG = LogisticRegression(solver='liblinear',max_iter=200, C=0.5,random_state=0,)
clfLinSVM = SVC(kernel='linear',random_state=0, probability=True,gamma=1)
clfSVC = SVC(kernel='rbf',random_state=0, probability=True,gamma=1)
clfDT = DecisionTreeClassifier(random_state=0)
clfRFC = RandomForestClassifier(n_estimators=100, max_depth=4, random_state=0)
clfMnb = MultinomialNB()
clfBnb = BernoulliNB()
clfAB = AdaBoostClassifier(base_estimator=None, n_estimators=100, learning_rate=.1,  random_state=0)

columnas = ['Char-word-func-N-grams-combination','c','w','f',
                                  'total_feat_num',
                                  'weight scheme',
                                  'classifier',
                                  'cv-acc',
                                  'cv-balanced_acc',
                                  'cv-precision',
                                  'cv-recall',
                                  'cv-f1',
                                  'cv-roc_auc',
            
                                  'test-acc',
                                  'test-balanced_acc',
            
                                  'test-real-precision',
                                  'test-real-recall',
                                  'test-real-f1',
            
                                  'test-fake-precision',
                                  'test-fake-recall',
                                  'test-fake-f1',
            
                                  'test-roc_auc',
                                  'fpr',
                                  'tpr',
                                  'thresholds',
                                  'Status'
           
           ]
import collections
def check_model_fit(clf, X_test, y_test):
    # Print overall test-set accuracy
    y_pred = clf.predict(X_test)
    target_names = [1, 0]
    result = precision_recall_fscore_support(y_test, y_pred,average=None,labels=target_names)
    real_precision = result[0][0]
    fake_precision = result[0][1]
    real_recall = result[1][0]
    fake_recall = result[1][1]
    real_f1 = result[2][0]
    fake_f1 = result[2][1]
    
    acc = accuracy_score(y_test, y_pred,normalize=True)
    balanced_acc = balanced_accuracy_score(y_test, y_pred)   
    
    probs = clf.predict_proba(X_test)
    probs = probs[:, 1]
    roc_auc = roc_auc_score(y_test, probs)
    fpr, tpr, thresholds = roc_curve(y_test, probs)
        
    results = {'accuracy': acc, 
               'balanced_accuracy': balanced_acc, 
               'real_precision': real_precision, 
               'real_recall': real_recall, 
               'real_f1': real_f1,
               'fake_precision': fake_precision, 
               'fake_recall': fake_recall, 
               'fake_f1': fake_f1, 
               'roc_auc': roc_auc,
               'fpr': fpr,
               'tpr': tpr,
               'thresholds': 1
              }
    
    return results

def cv(clf, train_X, train_y, scoring=['accuracy', 'balanced_accuracy' ,'precision', 'recall', 'f1', 'roc_auc']): 
    
    cv = cross_validate(clf, train_X, train_y, cv=5, return_estimator=True, scoring=scoring)
    output = {}
    for s in scoring: 
        m = np.mean(cv['test_' + s])
        v = np.var(cv['test_' + s])
        output[s] ={'mean': m, 'var': v}  
    return output

def end_to_end(train_texts, train_labels_dict, test_texts, test_labels_dict, 
             ngram_feat_sets=[[[1]],[[0]],[[0]]], weight_schemes=['binary'], clfs=[clfLG], 
             expLog_df=pd.DataFrame(columns=['N-gram-feat-comb', 'char N-gram', 'word N-gram', 'func N-gram'  , 'total_feat_num', 'weight scheme', 'classifier', 'cv-acc', 'cv-balanced_acc','cv-roc', 'cv-f1-fake', 'acc', 'balanced_acc', 'roc', 'f1_fake', 'f1_real']), 
             name_prefix=''):  
    
    x = 0
    start = time.time()
    errors = 0
    total = 0
    div = 100
    for i in range(len(ngram_feat_sets)): 
        permutation = ngram_feat_sets[i] 
        cnvalues = permutation[0]
        wnvalues = permutation[1] 
        fnvalues = permutation[2] 
        ngram_feat_type_set_str = str(cnvalues) +'-' + str(wnvalues) + '-' + str(fnvalues) + ' char-word-func'
        char_n_gram = cnvalues
        word_n_gram = wnvalues
        func_n_gram = fnvalues
        
        feat_extractor = FeatureExtractor(cnvalues, wnvalues, fnvalues)        
        train_feature_mx, _ = feat_extractor.fit_extract(train_texts)
        test_feature_mx, _ = feat_extractor.extract(test_texts) 
        
        #applying the frequency threshold
        cols_to_keep = feat_extractor.apply_frequency_threshold(train_feature_mx)
        thresholded_train_feat_mx = train_feature_mx[:, cols_to_keep]
        thresholded_test_feat_mx = test_feature_mx[:, cols_to_keep]

        
        for weight_scheme_name in weight_schemes: 
            
            weight_scheme = get_weighting_scheme(weight_scheme_name)
            for clf in clfs: 
                
                clf_name = type(clf).__name__
                #print('Classifier: ' + clf_name)
                ppl = Pipeline([('scaler', weight_scheme), ('clf', clf)]) 
                row = None
                try:
                    train_X = thresholded_train_feat_mx.astype('float')
                    train_y = train_labels_dict.values.ravel().astype('int')
                    test_X = thresholded_test_feat_mx.astype('float')
                    test_y = test_labels_dict.values.ravel().astype('int')
                    cv_scores = cv(ppl, train_X, train_y)

                    ppl.fit(train_X, train_y)
                    test_scores = check_model_fit(ppl, test_X, test_y)
                    row = [name_prefix + ngram_feat_type_set_str,
                    cnvalues,
                    wnvalues, 
                    fnvalues,        
                    thresholded_train_feat_mx.shape[1],
                    weight_scheme_name, 
                    clf_name, 
                    np.round(cv_scores['accuracy']['mean'], 2),
                    np.round(cv_scores['balanced_accuracy']['mean'], 2),
                    np.round(cv_scores['precision']['mean'], 2),
                    np.round(cv_scores['recall']['mean'], 2),
                    np.round(cv_scores['f1']['mean'], 2),
                    np.round(cv_scores['roc_auc']['mean'], 2),
                           
                    np.round(test_scores['accuracy'], 2),
                    np.round(test_scores['balanced_accuracy'], 2),
                           
                    np.round(test_scores['real_precision'], 2),
                    np.round(test_scores['real_recall'], 2),
                    np.round(test_scores['real_f1'], 2),
                           
                    np.round(test_scores['fake_precision'], 2),
                    np.round(test_scores['fake_recall'], 2),
                    np.round(test_scores['fake_f1'], 2),
                           
                    np.round(test_scores['roc_auc'], 2),
                    test_scores['tpr'],
                    test_scores['fpr'], 
                    test_scores['thresholds'],              
                     "OK"     
                          ]
                    expLog_df.loc[len(expLog_df),] = row
                except Exception as e:
                    errors = errors + 1
                    total = total + 1
                    row = [name_prefix + ngram_feat_type_set_str,
                    thresholded_train_feat_mx.shape[1], 
                    weight_scheme_name, 
                    clf_name, 
                    None,
                    None,
                    None,
                    None,
                    None,
                    None,                 
                    None,
                    None,
                    None,
                    None,
                    None,
                    None,
                    None,
                    None,
                    None,
                    None,
                    None,
                    None,
                    None,
                    None,
                    None,
                    e    ]
                    expLog_df.loc[len(expLog_df),] = row
                if(x>0 and x%div==0):
                    end = time.time()
                    print("i:{i:1d}, Tiempo : {h:1d} h {m:1d} m {s:1.1f} s".format(i=x,h=int((end - start)/3600),m=int(((end - start)%3600)/60),s=(((end - start)%3600)%60)))
                    print("Last run - Correct: {c:1d} Fail: {f:1d} | Total - Correct {tc:1d} Fail: {tf:1d}".format(c=div-errors,f=errors,tc=x-total,tf=total))
                    print(row)
                    print("_________________________________________________________________________________")
                    errors = 0
                    expLog_df.to_csv(Path("results/results"+str(x)+".csv"), index=False)

                x=x+1
    i = i + 1
    return expLog_df

In [ ]:

my_expLog = pd.DataFrame(columns=columnas)
my_ngram_feat_combinations = ngram_feat_combinations
# specify the weighting schemes you want to use from the list: 
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
#ttps://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
# weighting_schemes = ['binary', 'logent', 'tfidf', 'norm', 'relat', 'none (tf)']
my_weighting_schemes = ['binary', 'logent', 'tfidf', 'norm', 'relat', 'none (tf)']
# specify the classifiers you want to use form the list:  
# [clfLG, clfLinSVM, clfSVC, clfDT, clfRFC, clfMnb, clfBnb,  clfAB]
my_clfs = [clfLG, clfLinSVM, clfSVC, clfDT, clfRFC, clfMnb, clfBnb,  clfAB]

# run the experiments
expLog = end_to_end(xTrain['Text'],
                    yTrain,
                    xTest['Text'],
                    yTest,
                    my_ngram_feat_combinations, 
                    my_weighting_schemes, 
                    my_clfs,
                    my_expLog)
expLog.to_csv(Path('results/results.csv'), index=False)
expLog

i:100, Tiempo : 0 h 5 m 37.7 s
Last run - Correct: 84 Fail: 16 | Total - Correct 84 Fail: 16
['0-0-3 char-word-func', 0, 0, 3, 3476, 'binary', 'RandomForestClassifier', 0.56, 0.5, 0.56, 0.98, 0.71, 0.44, 0.54, 0.5, 0.55, 0.97, 0.7, 0.44, 0.03, 0.05, 0.48, array([0.        , 0.00581395, 0.00581395, 0.01162791, 0.01162791,
       0.01744186, 0.01744186, 0.02906977, 0.02906977, 0.04069767,
       0.04069767, 0.04651163, 0.04651163, 0.05813953, 0.05813953,
       0.06395349, 0.06395349, 0.0755814 , 0.0755814 , 0.09302326,
       0.09302326, 0.11046512, 0.11046512, 0.12790698, 0.12790698,
       0.13372093, 0.13372093, 0.15697674, 0.15697674, 0.1627907 ,
       0.1627907 , 0.16860465, 0.16860465, 0.1744186 , 0.22674419,
       0.22674419, 0.23255814, 0.23255814, 0.23837209, 0.23837209,
       0.24418605, 0.24418605, 0.25      , 0.25      , 0.25581395,
       0.25581395, 0.29651163, 0.29651163, 0.30232558, 0.31395349,
       0.31976744, 0.31976744, 0.3255814 , 0.3255814 , 0.33139535,
       

i:400, Tiempo : 0 h 22 m 28.3 s
Last run - Correct: 84 Fail: 16 | Total - Correct 336 Fail: 64
['0-1-2 char-word-func', 0, 1, 2, 7024, 'tfidf', 'LogisticRegression', 0.55, 0.5, 0.56, 0.94, 0.7, 0.47, 0.55, 0.51, 0.55, 0.93, 0.69, 0.52, 0.09, 0.15, 0.52, array([0.        , 0.00581395, 0.09302326, 0.09302326, 0.09883721,
       0.09883721, 0.11046512, 0.11046512, 0.12209302, 0.12209302,
       0.14534884, 0.14534884, 0.15116279, 0.15116279, 0.16860465,
       0.16860465, 0.19186047, 0.19186047, 0.19767442, 0.19767442,
       0.20348837, 0.20348837, 0.23837209, 0.23837209, 0.24418605,
       0.24418605, 0.25581395, 0.25581395, 0.26162791, 0.26162791,
       0.26744186, 0.26744186, 0.27325581, 0.27325581, 0.27906977,
       0.27906977, 0.28488372, 0.28488372, 0.30813953, 0.30813953,
       0.3255814 , 0.3255814 , 0.33139535, 0.33139535, 0.3372093 ,
       0.3372093 , 0.34883721, 0.34883721, 0.38372093, 0.38372093,
       0.40116279, 0.40116279, 0.40697674, 0.40697674, 0.4127907 ,
       0.

i:600, Tiempo : 0 h 43 m 24.3 s
Last run - Correct: 84 Fail: 16 | Total - Correct 504 Fail: 96
['0-1-6 char-word-func', 0, 1, 6, 3942, 'norm', 'LogisticRegression', 0.55, 0.54, 0.59, 0.62, 0.61, 0.57, 0.55, 0.54, 0.59, 0.59, 0.59, 0.5, 0.5, 0.5, 0.61, array([0.        , 0.00581395, 0.0872093 , 0.09883721, 0.10465116,
       0.10465116, 0.12790698, 0.12790698, 0.13372093, 0.13372093,
       0.14534884, 0.14534884, 0.15697674, 0.15697674, 0.16860465,
       0.16860465, 0.1744186 , 0.1744186 , 0.18023256, 0.18023256,
       0.22093023, 0.22093023, 0.22674419, 0.22674419, 0.25      ,
       0.25      , 0.25581395, 0.25581395, 0.29651163, 0.29651163,
       0.31976744, 0.31976744, 0.3255814 , 0.3255814 , 0.33139535,
       0.33139535, 0.3372093 , 0.3372093 , 0.34302326, 0.34302326,
       0.34883721, 0.34883721, 0.35465116, 0.35465116, 0.37790698,
       0.37790698, 0.38372093, 0.38372093, 0.38953488, 0.38953488,
       0.39534884, 0.39534884, 0.4127907 , 0.4127907 , 0.4244186 ,
       0.42

i:800, Tiempo : 1 h 4 m 8.8 s
Last run - Correct: 83 Fail: 17 | Total - Correct 671 Fail: 129
['0-2-3 char-word-func', 9426, 'relat', 'LogisticRegression', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, ValueError('The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()')]
_________________________________________________________________________________
i:900, Tiempo : 1 h 11 m 24.1 s
Last run - Correct: 80 Fail: 20 | Total - Correct 751 Fail: 149
['0-2-5 char-word-func', 6047, 'relat', 'RandomForestClassifier', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, ValueError('The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()')]
_________________________________________________________________________________
i:1000, Tiempo : 1 h 16 m 19.2 s
Last run - Correct:

i:1200, Tiempo : 1 h 27 m 55.2 s
Last run - Correct: 84 Fail: 16 | Total - Correct 1000 Fail: 200
['0-3-5 char-word-func', 0, 3, 5, 2409, 'binary', 'LogisticRegression', 0.51, 0.49, 0.55, 0.6, 0.58, 0.49, 0.56, 0.55, 0.59, 0.67, 0.62, 0.52, 0.43, 0.47, 0.54, array([0.        , 0.00581395, 0.06976744, 0.06976744, 0.08139535,
       0.08139535, 0.09883721, 0.09883721, 0.11046512, 0.11046512,
       0.15697674, 0.15697674, 0.1627907 , 0.1627907 , 0.16860465,
       0.16860465, 0.1744186 , 0.1744186 , 0.19186047, 0.19186047,
       0.19767442, 0.19767442, 0.20348837, 0.20930233, 0.20930233,
       0.23255814, 0.23255814, 0.23837209, 0.23837209, 0.24418605,
       0.24418605, 0.26162791, 0.26162791, 0.31395349, 0.31395349,
       0.31976744, 0.31976744, 0.3372093 , 0.3372093 , 0.34302326,
       0.34302326, 0.35465116, 0.35465116, 0.36627907, 0.36627907,
       0.37209302, 0.37209302, 0.37790698, 0.37790698, 0.38372093,
       0.38372093, 0.38953488, 0.38953488, 0.39534884, 0.39534884,
    

...

i:5300, Tiempo : 7 h 42 m 37.8 s
Last run - Correct: 84 Fail: 16 | Total - Correct 4420 Fail: 880
['2-1-6 char-word-func', 2, 1, 6, 5608, 'tfidf', 'RandomForestClassifier', 0.69, 0.65, 0.65, 0.99, 0.78, 0.82, 0.68, 0.64, 0.63, 1.0, 0.77, 1.0, 0.29, 0.45, 0.87, array([0.        , 0.00581395, 0.05232558, 0.06395349, 0.39534884,
       0.39534884, 0.41860465, 0.41860465, 0.43023256, 0.43023256,
       0.48255814, 0.48255814, 0.48837209, 0.48837209, 0.50581395,
       0.50581395, 0.51744186, 0.51744186, 0.52325581, 0.52325581,
       0.54651163, 0.54651163, 0.56976744, 0.56976744, 0.61046512,
       0.61046512, 0.61627907, 0.61627907, 0.62209302, 0.62209302,
       0.62790698, 0.62790698, 0.65116279, 0.65116279, 0.65697674,
       0.65697674, 0.70930233, 0.70930233, 0.72093023, 0.72093023,
       0.73255814, 0.73255814, 0.74418605, 0.74418605, 0.79651163,
       0.79651163, 0.80813953, 0.80813953, 0.81976744, 0.81976744,
       0.83139535, 0.84302326, 0.84302326, 0.87209302, 0.87209302,
  

i:5600, Tiempo : 8 h 57 m 37.6 s
Last run - Correct: 83 Fail: 17 | Total - Correct 4671 Fail: 929
['2-2-5 char-word-func', 7713, 'relat', 'LogisticRegression', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, ValueError('The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()')]
_________________________________________________________________________________
i:5700, Tiempo : 9 h 17 m 54.0 s
Last run - Correct: 80 Fail: 20 | Total - Correct 4751 Fail: 949
['2-3-0 char-word-func', 3978, 'relat', 'RandomForestClassifier', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, ValueError('The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()')]
_________________________________________________________________________________
i:5800, Tiempo : 9 h 39 m 48.2 s
Last run - Co

i:6100, Tiempo : 10 h 37 m 34.5 s
Last run - Correct: 84 Fail: 16 | Total - Correct 5084 Fail: 1016
['2-4-2 char-word-func', 2, 4, 2, 5369, 'binary', 'RandomForestClassifier', 0.71, 0.67, 0.66, 1.0, 0.79, 0.9, 0.67, 0.64, 0.62, 1.0, 0.77, 1.0, 0.27, 0.43, 0.95, array([0.        , 0.00581395, 0.03488372, 0.04651163, 0.49418605,
       0.49418605, 0.53488372, 0.53488372, 0.54651163, 0.54651163,
       0.55232558, 0.56395349, 0.59883721, 0.59883721, 0.64534884,
       0.64534884, 0.6744186 , 0.6744186 , 0.68604651, 0.68604651,
       0.70930233, 0.70930233, 0.72093023, 0.72093023, 0.73255814,
       0.73255814, 0.73837209, 0.73837209, 0.76162791, 0.76162791,
       0.78488372, 0.78488372, 0.80232558, 0.80232558, 0.80813953,
       0.80813953, 0.81395349, 0.81395349, 0.81976744, 0.81976744,
       0.83139535, 0.83139535, 0.87790698, 0.87790698, 0.90697674,
       0.90697674, 0.91860465, 0.91860465, 0.94767442, 0.94767442,
       0.95348837, 0.95348837, 0.95930233, 0.95930233, 0.97093023,
 

i:6500, Tiempo : 11 h 56 m 21.6 s
Last run - Correct: 84 Fail: 16 | Total - Correct 5420 Fail: 1080
['2-5-3 char-word-func', 2, 5, 3, 5312, 'tfidf', 'RandomForestClassifier', 0.71, 0.67, 0.66, 1.0, 0.79, 0.87, 0.68, 0.64, 0.63, 1.0, 0.77, 1.0, 0.29, 0.45, 0.9, array([0.        , 0.00581395, 0.06976744, 0.08139535, 0.3372093 ,
       0.3372093 , 0.44767442, 0.44767442, 0.45348837, 0.45348837,
       0.46511628, 0.46511628, 0.51162791, 0.51162791, 0.51744186,
       0.51744186, 0.52906977, 0.52906977, 0.53488372, 0.53488372,
       0.55813953, 0.55813953, 0.5755814 , 0.5755814 , 0.62790698,
       0.62790698, 0.63953488, 0.63953488, 0.64534884, 0.65697674,
       0.6627907 , 0.6627907 , 0.66860465, 0.66860465, 0.6744186 ,
       0.6744186 , 0.70930233, 0.70930233, 0.72674419, 0.72674419,
       0.73837209, 0.73837209, 0.77325581, 0.77325581, 0.79069767,
       0.79069767, 0.79651163, 0.79651163, 0.80813953, 0.80813953,
       0.81976744, 0.81976744, 0.85465116, 0.85465116, 0.86046512,
  

i:6800, Tiempo : 12 h 49 m 22.3 s
Last run - Correct: 83 Fail: 17 | Total - Correct 5671 Fail: 1129
['2-6-2 char-word-func', 4809, 'relat', 'LogisticRegression', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, ValueError('The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()')]
_________________________________________________________________________________
i:6900, Tiempo : 13 h 7 m 36.4 s
Last run - Correct: 80 Fail: 20 | Total - Correct 5751 Fail: 1149
['2-6-4 char-word-func', 2550, 'relat', 'RandomForestClassifier', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, ValueError('The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()')]
_________________________________________________________________________________
i:7000, Tiempo : 13 h 23 m 8.5 s
Last run -

In [ ]:
# writing the Experiment Log to a .csv file (to be read as a DataFrame for analysis in a separate notebook)
expLog.to_csv(Path('results/results.csv'), index=False)